In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import requests
import pandas as pd
from os import getcwd
import urllib3
from bs4 import BeautifulSoup
from typing import List
from tqdm import tqdm
from time import sleep
import datetime as dt
from random import randint
from time import strptime, mktime, time

In [4]:
PROJECT_NAME = "wsj_tweet_scrapping"
PROJECT_PATH = getcwd()[: getcwd().find(PROJECT_NAME) + len(PROJECT_NAME)]
DATA_PATH = f"{PROJECT_PATH}\\data"
EXCEL_PATH = f"{DATA_PATH}\\excel"

In [5]:
def extract_text_from_url(url):
    """Extract text from the URL."""
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text)
    http = urllib3.PoolManager()
    response = http.request(method="GET", url=url)
    soup = BeautifulSoup(response.data)
    complete_texts = ""
    for text in soup.find_all(name="p"):
        complete_texts += " " + text.text
    return complete_texts

In [6]:
def clean_text(text):
    """Clean input text."""
    discard_texts = ["\n", "\t", "\r", "\xa0"] 
    for word in discard_texts: 
        text = text.replace(word, " ")
    text = " ".join(text.split())
    return text

In [7]:
def extract_urls_from_tweet_source(text_soup : str):
    soup = BeautifulSoup(text_soup)
    urls = []
    for url in soup.find_all(name="a", attrs={"data-url":True}):
        url = url["data-url"]
        if "wsj" in url:
            urls.append(url)
    return urls

In [9]:
failed_downloading_urls = []

In [16]:
dates = pd.date_range(start=dt.date(2009,1,1), end=dt.date(2011,11,1), freq=pd.offsets.MonthBegin(1))
dates = dates.sort_values(ascending=False)

In [18]:
t = tqdm(range(len(dates)))
for i in t:
    year = dates[i].year
    month = dates[i].month
    t.set_description(f"Iter : {i}, year : {year}, month : {month}")
    
    # reading raw tweets
    file_name = f"twint_wsj_logistics_tweets_{year}_{str(month).rjust(2,'0')}"
    monthly_tweets = pd.read_excel(f"{EXCEL_PATH}\\{file_name}.xlsx")
    
    if  len(monthly_tweets) == 0:
        continue
        
    # processing raw tweets
    processed_tweets = []
    for i in range(len(monthly_tweets)):
        tweet = {}
        tweet["data-item-id"] = monthly_tweets["data-item-id"].iloc[i]
        tweet["data-conversation-id"] = monthly_tweets["data-conversation-id"].iloc[i]
        tweet["date"] = monthly_tweets["date"].iloc[i]
        tweet["tweet"] = monthly_tweets["tweet"].iloc[i]
        for url in extract_urls_from_tweet_source(monthly_tweets["all-data"].iloc[i]):
            tweet["url"] = url
            processed_tweets.append(tweet)
    processed_tweets = pd.DataFrame(processed_tweets)
    processed_tweets = processed_tweets.drop_duplicates().reset_index(drop = True)
#     print(f"After processing : {len(processed_tweets)} entries")
    
    if len(processed_tweets) == 0:
        continue
        
    # fetching news articles
    monthly_texts = []
    for url in tqdm(processed_tweets["url"].values):
        try:
            text = extract_text_from_url(url)
            text = clean_text(text)
            text = {"url": url, "text": text}
            monthly_texts.append(text) 
            sleep(2)
        except Exception as e:
            failed_downloading_urls.append([year, month, url])
            print(f"failed extracting news, exception : {e}")
            
    
    # saving news article
    monthly_texts = pd.DataFrame(monthly_texts) 
    monthly_texts = pd.merge(processed_tweets, monthly_texts, on="url", how="left")
#     print(f"After extracting texts : {len(monthly_texts)} entries")
    
    file_name = f"twint_wsj_logistics_news_{year}_{str(month).rjust(2,'0')}"
    monthly_texts.drop_duplicates().to_excel(f"{EXCEL_PATH}\\{file_name}.xlsx", index=False)

 25%|████████████████████▊                                                              | 8/32 [00:28<01:04,  2.69s/it]

failed extracting news, exception : HTTPConnectionPool(host='www.wsj.com', port=80): Max retries exceeded with url: http://www.wsj.com/news/world/asia (Caused by ResponseError('too many redirects'))



 21%|█████████████████▌                                                                 | 7/33 [00:20<01:05,  2.53s/it]

failed extracting news, exception : HTTPSConnectionPool(host='www.wsj.com', port=443): Max retries exceeded with url: https://www.wsj.com/asia (Caused by ResponseError('too many redirects'))



Iter : 34, year : 2009, month : 1: 100%|███████████████████████████████████████████████| 35/35 [08:35<00:00, 14.72s/it]


In [12]:
pd.DataFrame(failed_downloading_urls, columns=["year", "month", "url"]).to_excel(f"{EXCEL_PATH}\\failed_download_news_url-{int(time())}.xlsx",
                                                                                 index=False) 